# Предсказание сложности английского текста

In [1]:
!pip install nltk

In [2]:
!pip install textstat

In [3]:
!python -m spacy download en


⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
!pip install spacy

In [5]:
import pandas as pd
import os
import glob
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import spacy
from spacy.lang.en import English
from sklearn.utils.class_weight import compute_class_weight
from textstat import textstat
import en_core_web_sm 
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from collections import OrderedDict
from sklearn.preprocessing import LabelEncoder
import numpy as np
from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

In [6]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
pd.set_option('display.max_columns', None)

### Начнем с загрузки данных

In [9]:
levels = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']

df1 = pd.DataFrame()  # Создаем пустой датафрейм


folder_path = "English_scores/Subtitles_all/Subtitles"  # Замените на путь к нужной папке

    # Получаем список всех файлов в папке
file_names = os.listdir(folder_path)[1:]


for filename in file_names:
            with open("English_scores/Subtitles_all/Subtitles/" + filename, 'r', encoding='latin-1') as f:  # Открываем файл по полному пути
                file_content = f.read()  # Читаем содержимое файла

                # Создаем новую строку в датасете с содержимым файла
                new_row = pd.DataFrame({'Subtitles': [file_content],
                                       'Movie':[filename]})
                df1 = pd.concat([df1, new_row], ignore_index=True)

def r(f):
    f = f.replace('.srt', '')
    return f
df1['Movie'] = df1['Movie'].apply(r)

df = pd.read_excel('English_scores/movies_labels.xlsx')
df3 = pd.merge(df, df1, on =  'Movie')



folder_path = "English_scores/Subtitles_all/Subtitles"  # Замените на путь к нужной папке
for level in levels[1: -2]:
        path = "English_scores/Subtitles_all/" + level
        file_names = os.listdir(path)
        for file in file_names:
            with open(path + '/' + file, 'r', encoding='latin-1') as f:  # Открываем файл по полному пути
                file_content = f.read()  # Читаем содержимое файла

                # Создаем новую строку в датасете с содержимым файла
                new_row = pd.DataFrame({'Subtitles': [file_content],
                                       'Level':[level]})
                df3= pd.concat([df3, new_row], ignore_index=True)
df3 = df3.drop(['Movie', 'id'], axis = 1)



for level in levels:
        path = "data" + '/' + level
        file_names = os.listdir(path)
        for file in file_names:
            with open(path + '/' + file, 'r', encoding='latin-1') as f:  # Открываем файл по полному пути
                file_content = f.read()  # Читаем содержимое файла

                # Создаем новую строку в датасете с содержимым файла
                new_row = pd.DataFrame({'Subtitles': [file_content],
                                       'Level':[level]})
                df3= pd.concat([df3, new_row], ignore_index=True)
df = df3.copy()
df

,Level,Subtitles
0,B1,"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,B1,"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,B2,"1\n00:00:15,089 --> 00:00:21,229\nResync: Xenz..."
3,B2,"1\n00:00:17,610 --> 00:00:22,610\n- <i><font c..."
4,A2/A2+,"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
...,...,...
1729,C2,ï»¿SeaWorld has suffered an 84% collapse in pr...
1730,C2,ï»¿They may not know who Steve Jobs was or eve...
1731,C2,"ï»¿A long time ago, cinema audiences were tran..."
1732,C2,ï»¿We may not yet be living in an age of flyin...


In [10]:
ad = pd.read_csv('cefr_leveled_texts.csv')

ad.rename(columns={'text': 'Subtitles', 'label': 'Level'}, inplace=True)

In [11]:
df = df.append(ad, ignore_index = True)

<ipython-input-11-0df02eed7470>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(ad, ignore_index = True)


In [50]:
df

,Level,Subtitles
0,B1,"1\n00:00:55,279 --> 00:01:07,279\n<font color=..."
1,B1,"1\n00:01:54,281 --> 00:01:55,698\nHey!\n\n2\n0..."
2,B2,"1\n00:00:15,089 --> 00:00:21,229\nResync: Xenz..."
3,B2,"1\n00:00:17,610 --> 00:00:22,610\n- <i><font c..."
4,A2/A2+,"1\n00:00:27,240 --> 00:00:30,879\n<i>Oh, I com..."
...,...,...
3223,C2,Light propagating in the vicinity of astrophys...
3224,C2,Future of dentistry has become one of the most...
3225,C2,﻿The forests – and suburbs – of Europe are ech...
3226,C2,Hedge funds are turning bullish on oil once ag...


In [12]:
df.Level.value_counts()

B2            708
A1            576
A2            550
C1            488
B1            464
C2            404
A2/A2+         25
B1, B2          8
A2/A2+, B1      5
Name: Level, dtype: int64

## Feature engineering

In [24]:
nlp = en_core_web_sm.load()

POS_TAGS = [
    "ADJ",
    "ADP",
    "ADV",
    "AUX",
    "CONJ",
    "CCONJ",
    "DET",
    "INTJ",
    "NOUN",
    "NUM",
    "PART",
    "PRON",
    "PROPN",
    "PUNCT",
    "SCONJ",
    "SYM",
    "VERB",
    "X",
    "SPACE",
]


def generate_features(data):
    """Generate features for a list of texts

    Args:
        data (list[str]): the dataset to be processed.

    Returns:
        pandas.DataFrame: the processed features.
    """
    feature_data = []

    for text in data:
        
        features = preprocess_text(text)
        feature_data.append(features)
    
    return pd.DataFrame(feature_data)


def preprocess_text(text):
    """Takes a text, generate features, and returns as dict

    Args:
        text (str): the text to be preprocessed.

    Returns:
        dict: a dictionary of feature names with associated values

    """
    
    text = _simplify_punctuation(text)
#     text = _remove_stop_words(text)
#     text = _stem(text)

    features = {
#         "words_count": len(text.split()),
#         "symbols_count": len(text),
        "flesch_reading_ease": textstat.flesch_reading_ease(text),
        
#         "textstat.syllable_count":textstat.syllable_count(text),
#         "lexicon_count":textstat.lexicon_count(text, removepunct=True),
#         "sentence_count":textstat.sentence_count(text),
#         "polysyllabcount":textstat.polysyllabcount(text),
#         "monosyllabcount":textstat.monosyllabcount(text),
        
        "spache_readability":textstat.spache_readability(text),
        "mcalpine_eflaw":textstat.mcalpine_eflaw(text),
        "smog_index": textstat.smog_index(text),
        "flesch_kincaid_grade": textstat.flesch_kincaid_grade(text),
        "coleman_liau_index": textstat.coleman_liau_index(text),
        "automated_readability_index": textstat.automated_readability_index(text),
        "dale_chall_readability_score": textstat.dale_chall_readability_score(text),
        "difficult_words": textstat.difficult_words(text) / len(text.split()),
        "linsear_write_formula": textstat.linsear_write_formula(text),
        "gunning_fog": textstat.gunning_fog(text),
        "text_standard": textstat.text_standard(text, float_output=True),
        "mean_parse_tree_depth": get_mean_parse_tree_depth(text),
        "mean_ents_per_sentence": get_mean_ents_per_sentence(text),
    }

    features.update(get_mean_pos_tags(text))

    return features


def _simplify_punctuation(text):
        
        text = text.replace('\ufeff', '')
        text = text.lower()
        text = re.sub(r"[,:;()\-]", " ", text)  # Override commas, colons, etc to spaces/
        text = re.sub(r"[\.!?]", ".", text)  # Change all terminators like ! and ? to "."
        text = re.sub(r"^\s+", "", text)  # Remove white space
        text = re.sub(r"[ ]*(\n|\r\n|\r)[ ]*", " ", text)  # Remove new lines
        text = re.sub(r'\d+:\d+:\d+,\d+|\d+|<[^>]*>|-|–|>|%|[’\':;♪(){}\[\]]', '', text)  # Remove timecodes, digits, HTML tags, special characters, brackets, exclamation marks, question marks, colons, and other similar characters
        text = re.sub(r"([\.])[\. ]+", ".", text)  # Change all ".." to "."
        text = re.sub(r"[ ]*([\.])", ". ", text)  # Normalize all "."`
        text = re.sub(r"\s+", " ", text)  # Remove multiple spaces
        text = re.sub(r"\s+$", "", text)  # Remove trailing spaces
        text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
        text = text.replace('"', '')  # Remove smart quotes
        text = ''.join(char for char in text if ord(char) < 128)
        
        return text
    
    
# def _remove_stop_words(text):
#     # Tokenization - split text into individual words
#     words = word_tokenize(text)

#     # Remove stopwords
#     stop_words = set(stopwords.words('english'))
#     words = [word for word in words if word.lower() not in stop_words]

#     # Join the processed words back into a single string
#     processed_text = ' '.join(words)

#     return processed_text


# def _stem(text): 
#     stemmer = PorterStemmer()
    
#     # Tokenize the text into individual words
#     words = word_tokenize(text)
    
#     # Apply stemming to each word and store the results in a list
#     stemmed_words = [stemmer.stem(word) for word in words]
    
#     # Join the stemmed words back into a single string
#     stemmed_text = ' '.join(stemmed_words)
    
#     return stemmed_text

def get_mean_parse_tree_depth(text):
    """Calculate the average depth of parse trees in the text"""
    sentences = text.split(".")
    depths = []
    for doc in list(nlp.pipe(sentences)):
        depths += _get_parse_tree_depths(doc)
    return np.mean(depths)


def _get_parse_tree_depths(doc):
    return [_get_depth(token) for token in doc]


def _get_depth(token, depth=0):
    depths = [_get_depth(child, depth + 1) for child in token.children]
    return max(depths) if len(depths) > 0 else depth


def get_mean_pos_tags(text):
    """Calculate the mean for each type of POS tag in the text"""
    sentences = text.split(".")
    sentence_counts = _make_pos_tag_count_lists(sentences)
    num_sentences = textstat.sentence_count(text)
    mean_pos_tags = _calculate_mean_per_tag(sentence_counts, num_sentences)
    return mean_pos_tags


def _make_pos_tag_count_lists(sentences):
    sentence_counts = {}
    for doc in list(nlp.pipe(sentences)):
        pos_counts = _get_pos_tag_counts(doc)
        for key in pos_counts:
            if key in sentence_counts:
                sentence_counts[key].append(pos_counts[key])
            else:
                sentence_counts[key] = [pos_counts[key]]
    return sentence_counts


def _get_pos_tag_counts(doc):
    pos_counts = {}
    pos_tags = [token.pos_ for token in doc]
    for tag in pos_tags:
        if tag in pos_counts:
            pos_counts[tag] += 1
        else:
            pos_counts[tag] = 1
    return pos_counts


def _calculate_mean_per_tag(counts, num_sentences):
    mean_pos_tags = {f"mean_{tag.lower()}": 0 for tag in POS_TAGS}
    for key in counts:
        if len(counts[key]) < num_sentences:
            counts[key] += [0] * (num_sentences - len(counts[key]))
        mean_value = round(np.mean(counts[key]), 2)
        mean_pos_tags["mean_" + key.lower()] = mean_value
    return mean_pos_tags


def get_total_ents(text):
    """Get the total number of named entities in the text"""
    return len(nlp(text).doc.ents)


def get_mean_ents_per_sentence(text):
    """Calculate the average number of named entities per sentence in the text"""
    return get_total_ents(text) / textstat.sentence_count(text)

In [25]:
fd = generate_features(list(df.Subtitles.values))

fd

,flesch_reading_ease,spache_readability,mcalpine_eflaw,smog_index,flesch_kincaid_grade,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,mean_parse_tree_depth,mean_ents_per_sentence,mean_adj,mean_adp,mean_adv,mean_aux,mean_conj,mean_cconj,mean_det,mean_intj,mean_noun,mean_num,mean_part,mean_pron,mean_propn,mean_punct,mean_sconj,mean_sym,mean_verb,mean_x,mean_space
0,89.24,2.65,10.7,6.4,2.7,5.06,3.2,6.21,0.087191,3.750000,4.13,6.0,0.811132,0.171761,0.45,0.49,0.55,0.54,0,0.18,0.42,0.19,1.19,0.04,0.27,1.60,0.17,0.00,0.13,0.00,1.62,0.00,1.00
1,97.30,2.52,11.5,6.2,1.7,3.20,1.9,5.75,0.067151,3.153846,4.04,2.0,0.796791,0.175856,0.47,0.56,0.53,0.60,0,0.16,0.47,0.26,1.17,0.03,0.33,1.79,0.16,0.01,0.21,0.00,1.52,0.01,1.00
2,98.92,2.26,9.3,6.0,1.0,2.61,1.6,5.59,0.060573,1.941176,3.24,6.0,0.753422,0.254371,0.33,0.48,0.30,0.51,0,0.18,0.43,0.09,1.10,0.08,0.20,1.34,0.30,0.00,0.12,0.00,1.07,0.00,1.00
3,97.09,2.22,12.0,5.7,1.7,2.40,1.1,4.84,0.027238,5.222222,3.57,2.0,0.798168,0.139900,0.48,0.55,0.60,0.66,0,0.17,0.48,0.39,1.02,0.03,0.36,1.98,0.10,0.00,0.20,0.00,1.57,0.00,1.15
4,97.09,2.53,11.7,6.2,1.7,4.02,2.4,5.74,0.063413,58.000000,4.06,2.0,0.842644,0.262231,0.58,0.56,0.55,0.60,0,0.14,0.56,0.26,1.38,0.05,0.31,1.55,0.28,0.01,0.14,0.00,1.50,0.01,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3223,38.96,6.14,32.7,14.6,13.7,12.48,14.6,9.04,0.228169,13.000000,13.76,14.0,1.780488,0.000000,2.93,3.93,0.67,1.33,0,0.73,3.60,0.00,6.40,0.00,0.20,0.60,0.07,0.00,0.47,0.00,2.73,0.00,0.93
3224,32.33,6.19,29.0,15.5,14.2,15.95,16.8,9.93,0.263158,14.200000,14.80,15.0,1.457429,0.461538,2.77,2.65,0.73,0.88,0,0.92,2.12,0.04,6.89,0.15,0.27,0.73,0.35,0.00,0.23,0.00,2.65,0.00,1.00
3225,45.29,6.28,34.8,13.4,13.4,13.23,16.2,9.19,0.210818,20.000000,13.54,14.0,1.918954,1.535714,2.43,3.86,0.68,1.61,0,1.07,2.04,0.00,7.21,0.14,0.54,1.36,1.32,0.00,0.64,0.00,3.25,0.00,1.18
3226,61.26,5.19,24.9,12.2,9.3,11.26,11.3,9.40,0.202920,12.200000,10.59,12.0,1.230665,1.324324,1.68,2.51,0.68,0.86,0,0.54,1.32,0.00,5.18,0.14,0.38,0.46,1.65,0.00,0.14,0.11,2.32,0.16,1.06


In [27]:
# fd[levels] = df['Subtitles'].apply(f, o_dict = level_dict).apply(pd.Series)

fd['Level'] = df['Level']
fd['Level'] = fd['Level'].replace('A2/A2+, B1', 'A2')
fd['Level'] = fd['Level'].replace('B1, B2', 'B1')
fd['Level'] = fd['Level'].replace('A2/A2+', 'A2')

In [28]:
fd

,flesch_reading_ease,spache_readability,mcalpine_eflaw,smog_index,flesch_kincaid_grade,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,mean_parse_tree_depth,mean_ents_per_sentence,mean_adj,mean_adp,mean_adv,mean_aux,mean_conj,mean_cconj,mean_det,mean_intj,mean_noun,mean_num,mean_part,mean_pron,mean_propn,mean_punct,mean_sconj,mean_sym,mean_verb,mean_x,mean_space,Level
0,89.24,2.65,10.7,6.4,2.7,5.06,3.2,6.21,0.087191,3.750000,4.13,6.0,0.811132,0.171761,0.45,0.49,0.55,0.54,0,0.18,0.42,0.19,1.19,0.04,0.27,1.60,0.17,0.00,0.13,0.00,1.62,0.00,1.00,B1
1,97.30,2.52,11.5,6.2,1.7,3.20,1.9,5.75,0.067151,3.153846,4.04,2.0,0.796791,0.175856,0.47,0.56,0.53,0.60,0,0.16,0.47,0.26,1.17,0.03,0.33,1.79,0.16,0.01,0.21,0.00,1.52,0.01,1.00,B1
2,98.92,2.26,9.3,6.0,1.0,2.61,1.6,5.59,0.060573,1.941176,3.24,6.0,0.753422,0.254371,0.33,0.48,0.30,0.51,0,0.18,0.43,0.09,1.10,0.08,0.20,1.34,0.30,0.00,0.12,0.00,1.07,0.00,1.00,B2
3,97.09,2.22,12.0,5.7,1.7,2.40,1.1,4.84,0.027238,5.222222,3.57,2.0,0.798168,0.139900,0.48,0.55,0.60,0.66,0,0.17,0.48,0.39,1.02,0.03,0.36,1.98,0.10,0.00,0.20,0.00,1.57,0.00,1.15,B2
4,97.09,2.53,11.7,6.2,1.7,4.02,2.4,5.74,0.063413,58.000000,4.06,2.0,0.842644,0.262231,0.58,0.56,0.55,0.60,0,0.14,0.56,0.26,1.38,0.05,0.31,1.55,0.28,0.01,0.14,0.00,1.50,0.01,1.00,A2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3223,38.96,6.14,32.7,14.6,13.7,12.48,14.6,9.04,0.228169,13.000000,13.76,14.0,1.780488,0.000000,2.93,3.93,0.67,1.33,0,0.73,3.60,0.00,6.40,0.00,0.20,0.60,0.07,0.00,0.47,0.00,2.73,0.00,0.93,C2
3224,32.33,6.19,29.0,15.5,14.2,15.95,16.8,9.93,0.263158,14.200000,14.80,15.0,1.457429,0.461538,2.77,2.65,0.73,0.88,0,0.92,2.12,0.04,6.89,0.15,0.27,0.73,0.35,0.00,0.23,0.00,2.65,0.00,1.00,C2
3225,45.29,6.28,34.8,13.4,13.4,13.23,16.2,9.19,0.210818,20.000000,13.54,14.0,1.918954,1.535714,2.43,3.86,0.68,1.61,0,1.07,2.04,0.00,7.21,0.14,0.54,1.36,1.32,0.00,0.64,0.00,3.25,0.00,1.18,C2
3226,61.26,5.19,24.9,12.2,9.3,11.26,11.3,9.40,0.202920,12.200000,10.59,12.0,1.230665,1.324324,1.68,2.51,0.68,0.86,0,0.54,1.32,0.00,5.18,0.14,0.38,0.46,1.65,0.00,0.14,0.11,2.32,0.16,1.06,C2


In [29]:
fd.to_csv('datadata.csv')

## Получили признаки, на которых можем обучать модель

In [31]:
data = fd.copy().sample(frac = 1)
X = data.drop('Level', axis=1)
y = data['Level']

# encoder = LabelEncoder()
# y = encoder.fit_transform(y)

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# classes = np.unique(y_train)
# weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
# class_weights = dict(zip(classes, weights))


# Step 3: Create a CatBoost classifier and train it on the training data
# You can customize the hyperparameters based on your specific use case
catboost_model = CatBoostClassifier(iterations=40000,
#                                     cat_features = cat_features,# Number of boosting iterations
#                                     learning_rate=0.3,
                                     # Learning rate for boosting
#                                     depth=6,  # Depth of trees
                                    loss_function='MultiClass',  # For multi-class classification
#                                     eval_metric='MultiClass',  # Metric used for evaluation
                                    random_seed=42  # Random seed for reproducibility
#                                     class_weights = class_weights
                                    )

catboost_model.fit(X_train, y_train, verbose=10000, plot = True)  # Verbose setting shows progress every 100 iterations

# Step 4: Evaluate the model on the test data
accuracy = catboost_model.score(X_test, y_test)
print(f"Accuracy: {accuracy:.4f}")



MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.003678
0:	learn: 1.7865941	total: 191ms	remaining: 2h 7m 16s
10000:	learn: 0.2431344	total: 5m 35s	remaining: 16m 46s
20000:	learn: 0.1119593	total: 13m 11s	remaining: 13m 11s
30000:	learn: 0.0656743	total: 21m 13s	remaining: 7m 4s
39999:	learn: 0.0453426	total: 30m 40s	remaining: 0us
Accuracy: 0.9087


## Работа модели и ее результат

In [43]:
text = """Peter kicked his desk and stormed out of the classroom. Students in the rows next to him jumped, startled by the noise.

'Hey!' shouted Mr Clark, the maths teacher, to Peter's back as he continued down the corridor. 'Detention!' he called after him. Peter pushed his hands in his pockets, head down and shoulders up around his neck as if he was cold. It was the third time this month that he'd walked out of class.

***
Maria had forgotten her sports kit for class again. Last week, she'd avoided PE with a note from her mum. Her mother hadn't written it, of course. Maria had forged her mum's writing and hoped the sports teacher wouldn't notice. It was the second time she'd 'forgotten' her kit and towel this term. She was running out of excuses.

'Not good enough, Maria,' said her mother when the school called her to tell her Maria needed her kit. 'I put a clean towel on your bed this morning. I don't need the school calling and reminding me as if I'm a lazy mother. What's going on?'

'Nothing, Mum,' mumbled Maria. 

'Look at me and speak clearly,' said her mum, annoyed. 'I can't understand a word you're saying.'

Maria carried on looking down at the floor. At least her mum hadn't found the dress or the new trainers or the earrings – the presents guiltily hidden in her wardrobe. Then she would really be furious. And she'd start asking questions that Maria didn't want to answer.

'Fine. But if you don't tell me, I can't help you.'

***
A week later, Peter and Maria were sitting outside the head teacher's office. Peter was staring at the wall angrily as if he was silently arguing with it. Maria glanced at him and then quickly back at her bitten fingernails and then back at him again. She didn't need to wonder why he was here because she was in his class. She'd never spoken to him before, but she'd seen him walk out of class. Today, she'd seen him kick the desk so hard it hit the window and cracked the glass.

She wished she was here for something easy and straightforward like that. If she had damaged school property, it would be easy to apologise and promise never to do it again. She would pay for the damage and everyone would forget about it. Boys, especially teenage boys, were just aggressive sometimes, weren't they?

She imagined the conversation Peter would have inside the head's office.

'Why did you do that? Do you know how much furniture and windows cost?' Mr Hughes would say.

'Sorry. I was just angry with the teacher. I have a bad temper sometimes,' Peter would reply.

'It's not just me you need to apologise to. Here's the bill for the damage. Just don't do it again and the problem's solved, OK?'

Easy.

This simple solution of 'I'm sorry' and money wouldn't work for her. She was in trouble again about refusing to do PE. Her mother was on the way to the school, and Maria knew she was going to be asked questions that might lead to more questions. 

She watched Peter go in, slamming the door behind him. Even though she was sitting near the door, she couldn't hear any of their conversation. That meant they weren't shouting, so Peter was probably doing the apology part by now. After a few minutes, Peter left without looking at her. He didn't seem any less angry than before.

****
Maria didn't speak for eighteen minutes. She imagined herself floating above her mother's and Mr Hughes' heads, watching what was happening. Whenever they asked her a question, she lifted her shoulders in a silent shrug. It wasn't a great strategy, but it must have worked because they gave up eventually. It hadn't solved the real problem though. She could feel her phone vibrating inside her bag. She knew exactly who it was. Later she would have to answer his questions. Where had she been? Was she wearing the new dress? And detention wouldn't save her if he decided to wait outside school all night.

***
Peter and Maria were the only two students in detention on Monday. They were the only two on Tuesday too. They ignored each other on Monday, but exchanged nods on Tuesday. On Wednesday, Peter said, 'Hello again', when they arrived and Maria said, 'See you tomorrow' when they left. They both managed a smile at that.

On Thursday, Maria noticed bruises on Peter's stomach when he pulled his school bag over his head. They looked like they'd been there a while. He saw her staring and she looked away, pretending she hadn't seen anything. He pulled his shirt back down and his cheeks coloured. They didn't exchange words that day, but Maria felt as if, somehow, a kind of conversation had taken place anyway.

Maria wondered if Peter had someone he was afraid of. Someone who was nice at first and then later was like a different person. Someone who made him do things that made him feel uncomfortable. Someone who kept secrets and told him he had better be good at keeping secrets.

On Friday, she didn't bother putting her phone on silent. So what if the detention teacher saw the messages? Maybe that would be the beginning of the end of the nightmare. But the phone didn't make a sound. She deliberately pushed her sleeves up to above her elbows. She'd been so careful to hide her arms, but now the bruises were clearly on display. She remembered what her mother had said about helping her. She didn't say anything, but she hoped Peter would see them and hear the silent question. 'You too?'

***
Whatever Peter thought when he saw her arms, Maria didn't know because he left detention without a word. That night she waited until her mother was busy in the kitchen to talk to her.

She didn't know how to start. She practised different sentences in her head but she couldn't get the first word out of her mouth. 'Mum?' she said finally.

Her mother didn't look up from the vegetables she was preparing. 'Mmmm?'

'There's this boy at school and ...' Maria stopped. 'I saw something.'

'Saw what?' She had her mother's attention now.

'Something he didn't want me to see. A secret.'

'What kind of secret?' her mother said carefully.

'A bad secret – like I think someone is hurting him,' Maria said. But what if you tell someone and everyone thinks it's your fault? And what if you get someone in trouble and they get angry?' 

'Bad secrets are only bad until you tell someone,' her mother said. 'This boy needs to tell someone. 'But he has to choose the right person. A person who isn't going to say it's his fault, who's going to help.'

'Who is the right person?' asked Maria.

'An adult,' said her mother. 'One he trusts.'

Maria took a deep breath. She took her phone out of her bag and opened up the messages. The first word still wouldn't come. 'Mum?' she said finally. 'I have to tell you something …'"""

In [47]:
d = generate_features([text])

In [48]:
d

,flesch_reading_ease,spache_readability,mcalpine_eflaw,smog_index,flesch_kincaid_grade,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,mean_parse_tree_depth,mean_ents_per_sentence,mean_adj,mean_adp,mean_adv,mean_aux,mean_conj,mean_cconj,mean_det,mean_intj,mean_noun,mean_num,mean_part,mean_pron,mean_propn,mean_punct,mean_sconj,mean_sym,mean_verb,mean_x,mean_space
0,85.99,3.08,15.4,7.2,3.9,6.48,4.7,6.27,0.084661,5.0,5.39,5.0,0.926061,0.441441,0.48,0.97,0.6,0.86,0,0.41,0.74,0.03,1.84,0.03,0.47,2.04,0.38,0.14,0.32,0,1.91,0,1.0


In [49]:
catboost_model.predict(d)

array([['A2']], dtype=object)

In [22]:

import PyPDF2

# creating a pdf reader object


# print the number of pages in pdf file
def get_sl(reader):
    # print the text of the first page

    def create_dataset_with_levels(word_list):
        dataset = []
        levels = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
        current_level = None
        current_level_words = []
        sl = {}
        for word in word_list:
            if word in levels:
                # Found a level marker
                current_level = word
                current_level_words = []
                sl[word] = []
            elif word not in levels and '.' not in word and ',' not in word and current_level:
                # Associate the word with the current level
                sl[current_level].append(word)

        return sl
    
    
    
    l = []
    for i in range(len(reader.pages)):
        l.append(reader.pages[i].extract_text().split())

    m = []
    for i in range(len(l)):
        for j in range(len(l[i])):
            m.append(l[i][j])

            
    return create_dataset_with_levels(m)



reader = PyPDF2.PdfReader('Oxford_CEFR_level/American_Oxford_3000_by_CEFR_level.pdf')

d1 = get_sl(reader)

reader = PyPDF2.PdfReader('Oxford_CEFR_level/American_Oxford_5000_by_CEFR_level.pdf')

d2 = get_sl(reader)


reader = PyPDF2.PdfReader('Oxford_CEFR_level/The_Oxford_3000_by_CEFR_level.pdf')

d3 = get_sl(reader)


reader = PyPDF2.PdfReader('Oxford_CEFR_level/The_Oxford_5000_by_CEFR_level.pdf')


d4 = get_sl(reader)
dictm = {}

for d in [d1, d2, d3, d4]:
    for key, value in d.items():
        if key in dictm:
            dictm[key].extend(value)  # Если ключ уже есть в объединенном словаре, просто добавляем значения из текущего словаря
        else:
            dictm[key] = value[:]  # Если ключа еще нет, создаем новую запись в объединенном словаре



english_words_c2 = [
    "Aberration", "Abhorrent", "Abject", "Abscond", "Abstruse", "Accolade", "Acerbic", "Acquiesce", "Adamant",
    "Adulation", "Aesthetic", "Alacrity", "Alleviate", "Amalgamate", "Ambiguous", "Ameliorate", "Anachronism",
    "Anathema", "Animosity", "Antithesis", "Apotheosis", "Apprehensive", "Arcane", "Assiduous", "Assuage", "Audacious",
    "Auspicious", "Bellicose", "Benevolent", "Bombastic", "Cacophony", "Callous", "Capricious", "Cathartic",
    "Clandestine", "Coalesce", "Complacent", "Concomitant", "Confluence", "Conjecture", "Contrite", "Culpable",
    "Deleterious", "Despondent", "Diatribe", "Disparage", "Ebullient", "Ephemeral", "Epiphany", "Esoteric", "Euphoria",
    "Exacerbate", "Exhort", "Exonerate", "Facetious", "Fecund", "Garrulous", "Gregarious", "Hackneyed", "Harbinger",
    "Iconoclast", "Idiosyncratic", "Impetuous", "Ineffable", "Inscrutable", "Intrepid", "Intractable", "Inveterate",
    "Juxtapose", "Languid", "Loquacious", "Magnanimous", "Mellifluous", "Mendacious", "Mercurial", "Munificent",
    "Nefarious", "Nostalgia", "Obfuscate", "Obtuse", "Onerous", "Palliate", "Panacea", "Pariah", "Perfidious",
    "Perspicacious", "Platitude", "Pragmatic", "Prolific", "Pugnacious", "Querulous", "Quixotic", "Rancorous",
    "Rapport", "Recalcitrant", "Redolent", "Replete", "Reticent", "Salubrious", "Sycophant"
]
english_words_c2 += [
    "Sagacious", "Sanguine", "Solecism", "Soporific", "Specious", "Sycophant", "Trenchant", "Truncate",
    "Ubiquitous", "Unctuous", "Untenable", "Vacillate", "Vehement", "Venerate", "Veracity", "Verbose",
    "Vexation", "Vilify", "Voracious", "Wanderlust", "Xenophobia", "Yoke", "Zephyr", "Zealous"
]

english_words_c2 += [
    "Absolution", "Accretion", "Belligerent", "Cacophonous", "Debilitate", "Ebullience", "Exacerbation",
    "Facilitate", "Garrulity", "Hapless", "Iconoclastic", "Juxtaposition", "Kaleidoscope", "Labyrinthine",
    "Meticulous", "Nebulous", "Obfuscation", "Panegyric", "Pernicious", "Quintessential", "Recalcitrance",
    "Sycophantic", "Transcendence", "Ubiquity", "Vacuous", "Wistful", "Xenial", "Yield", "Zealotry"
]
english_words_c2 += [
    "Acrimonious", "Ambivalent", "Benevolence", "Capitulate", "Clandestine", "Cacophony", "Debunk", "Deleterious",
    "Disparate", "Elucidate", "Ephemeral", "Esoteric", "Euphemism", "Exacerbate", "Extemporaneous", "Fecund",
    "Fortuitous", "Gregarious", "Hackneyed", "Hapless", "Iconoclast", "Idiosyncratic", "Impetuous", "Ineffable",
    "Ingratiate", "Insidious", "Juxtapose", "Laconic", "Languid", "Mellifluous", "Mendacious", "Nefarious",
    "Ostentatious", "Palpable", "Perfidious", "Pernicious", "Platitude", "Pragmatic", "Quintessential", "Rancorous",
    "Sagacious", "Sanguine", "Sycophantic", "Trenchant", "Ubiquitous", "Untenable", "Vacillate", "Vehement",
    "Veracity", "Verbose", "Vexation", "Vilify", "Wanderlust", "Xenophobia", "Yoke", "Zephyr", "Zealous",
    "Absolution", "Accretion", "Belligerent", "Cacophonous", "Debilitate", "Ebullience", "Exacerbation",
    "Facilitate", "Garrulity", "Hapless", "Iconoclastic", "Juxtaposition", "Kaleidoscope", "Labyrinthine",
    "Meticulous", "Nebulous", "Obfuscation", "Panegyric", "Pernicious", "Quintessential", "Recalcitrance",
    "Sycophantic", "Transcendence", "Ubiquity", "Vacuous", "Wistful", "Xenial", "Yield", "Zealotry",
    "Aberrant", "Abhor", "Accolades", "Acrimony", "Adamantly", "Adversity", "Aesthetics", "Alacrity",
    "Alleviation", "Altruism", "Amicable", "Anomaly", "Antipathy", "Arduous", "Articulate", "Assimilate",
    "Audacity", "Augment", "Auspice", "Belligerence", "Blatant", "Bolster", "Cacophonic", "Camaraderie",
    "Capriciousness", "Catharsis", "Cogent", "Collaborative", "Comprehensive", "Concordant", "Conscientious",
    "Construe", "Contentious", "Contrition", "Convivial", "Cursory", "Debilitated", "Defunct", "Deleteriously",
    "Despondency", "Diligent", "Disconcerting", "Disparaging", "Duplicity", "Edification", "Effervescent",
    "Efficacious", "Eloquent", "Embellish", "Empathy", "Endeavor", "Enigmatic", "Equanimity", "Eradicate",
    "Erroneous", "Ethereal", "Exacerbating", "Exasperate", "Exemplary", "Expedient", "Expunge", "Extol",
    "Facilitation", "Fervent", "Flourish", "Fortitude", "Frivolous", "Garrulousness", "Gratuitous", "Guileless",
    "Harangue", "Heretic", "Hyperbole", "Hypothetical", "Immutable", "Impassioned", "Impeccable", "Imperturbable",
    "Inception", "Incorrigible", "Indifferent", "Inefficacious", "Infallible", "Ingenuity", "Innovative", "Insatiable",
    "Insipid", "Intrepidly", "Inundate", "Juxtaposed", "Kinetic", "Lamentation", "Laudatory", "Loquaciousness",
    "Lucid", "Magniloquent", "Malevolent", "Mellifluously", "Mendacity", "Meticulousness", "Mitigate", "Munificence",
    "Nefariously", "Nostalgic", "Ominous", "Opulent", "Panacea", "Pensive", "Perfunctory", "Phenomenon",
    "Plethora", "Ponderous", "Pristine", "Proclivity", "Prolificacy", "Propensity", "Prudent", "Puerile",
    "Quintessentially", "Quotidian", "Recalcitrant", "Redundant", "Replete", "Resilient", "Reticence", "Rhapsody",
    "Sagacity", "Sanguineous", "Sardonic", "Serenity", "Sinister", "Sophistry", "Soporifically", "Spurious",
    "Stagnant", "Sublime", "Surreptitious", "Sycophantically", "Taciturn", "Tantalizing", "Tempestuous", "Tendentious",
    "Tenuous", "Trepidation", "Ubiquitously", "Unassailable", "Unprecedented", "Unrequited", "Unscrupulous", "Untoward",
    "Uproarious", "Vacillation", "Vehemently", "Veritable", "Vexatious", "Vilification", "Vociferous", "Voraciously",
    "Wane", "Waver", "Xenophobia", "Yieldingly", "Zealously", "Zenith"
]

english_words_c2 += [
    "Aberrations", "Abhorrence", "Abjectly", "Absconding", "Abstruseness", "Acclamatory", "Acrimoniously", "Adamantness",
    "Adulations", "Aestheticism", "Alacritously", "Alleviations", "Amalgamations", "Ambiguities", "Ameliorations",
    "Anachronistic", "Anathematize", "Animosities", "Antagonistic", "Antipathies", "Aphoristically", "Apostates",
    "Approbation", "Archetypical", "Arduousness", "Articulately", "Assiduously", "Assuagements", "Auspiciously",
    "Belligerently", "Bellicosity", "Beneficently", "Bombastically", "Cacophonously", "Camaraderie", "Capitulation",
    "Clandestinely", "Collaboration", "Complaisantly", "Conciliatory", "Concordantly", "Conscientiousness",
    "Constrictive", "Contemporaneous", "Contiguously", "Contradiction", "Contrariness", "Convalescence",
    "Conventional", "Cosmopolitan", "Culpability", "Debilitating", "Debilitatingly", "Debunking", "Decorousness",
    "Decrepitude", "Defenestrate", "Deleteriously", "Deliberation", "Despondently", "Detestable", "Deviousness",
    "Diaphanous", "Didacticism", "Dilettantism", "Disaffection", "Disappointment", "Disparagements", "Dissonantly",
    "Dissolution", "Duplicities", "Edaciousness", "Edification", "Effervescently", "Efficaciously", "Effrontery",
    "Elaborations", "Elatedness", "Elucidation", "Embittered", "Emendations", "Encompassing", "Enervation",
    "Engrossingly", "Enigmaticness", "Equitableness", "Eradication", "Erroneously", "Estrangement", "Ethnological",
    "Euphemistically", "Exacerbated", "Exacerbatingly", "Exasperation", "Exemplification", "Exhaustiveness",
    "Exorbitantly", "Expediencies", "Expediently", "Expeditiously", "Exploitative", "Expurgation", "Extemporizing",
    "Facetiousness", "Factitiousness", "Facultatively", "Fallaciousness", "Fascinatingly", "Fecklessness",
    "Felicitation", "Fictitiously", "Flamboyantly", "Flauntingly", "Fluctuations", "Fortification", "Fortuitously",
    "Fulsomely", "Furtiveness", "Gallivanting", "Galvanizing", "Garishly", "Generosities", "Gratuitously",
    "Gregariously", "Hallowedness", "Harmoniously", "Harrowingly", "Histrionically", "Horticultural",
    "Hubristically", "Hypertrophied", "Hypnotically", "Idyllically", "Illegitimate", "Illustriously", "Imbecilic",
    "Immaculately", "Immethodical", "Immovability", "Impassiveness", "Impersonally", "Impracticably", "Impromptu",
    "Impudence", "Inappreciable", "Incapacitated", "Incessantly", "Incorrigibility", "Incredulously", "Incumbently",
    "Indefatigably", "Inefficaciously", "Inexorability", "Inexpugnable", "Infallibility", "Informatively", "Inimically",
    "Inopportunely", "Inscrutableness", "Insinuatingly", "Insipidness", "Insolubility", "Insolubly", "Instructively",
    "Insufferable", "Insupportably", "Intempestively", "Intentionality", "Interchangeably", "Interconnections",
    "Intergalactic", "Interminability", "Intimidation", "Intoxicating", "Intractability", "Intrepidness", "Intriguingly",
    "Inviolate", "Involuntarily", "Invulnerability", "Irrefutably", "Irresistible", "Jingoistically", "Joviality",
    "Judiciously", "Juxtapositions", "Kaleidoscopic", "Laconically", "Languorously", "Lapidary", "Lavishness",
    "Legitimately", "Levity", "Libertarian", "Lilliputian", "Lionhearted", "Loquaciousness", "Luminescent",
    "Lyricalness", "Machiavellian", "Mafioso", "Magnanimously", "Magniloquence", "Maladroit", "Malleability",
    "Manifestation", "Manifoldly", "Mannerliness", "Marvellously", "Mawkishness", "Meanderingly", "Melancholically",
    "Mellifluousness", "Memorability", "Mendaciously", "Mendicancy", "Metamorphosis", "Metaphorical", "Meticulousness",
    "Microcosmic", "Milquetoast", "Minacious", "Ministration", "Miraculousness", "Mitigating", "Moderateness",
    "Momentousness", "Monotonically", "Multifariously", "Multivalence", "Multitudinously", "Munificently",
    "Mutualities", "Myriad", "Mystification", "Mythological", "Nascent", "Nefariously", "Negotiability",
    "Neuroticism", "Neutralization", "Nimbleness", "Nomenclature", "Nonchalantly", "Nonetheless", "Notoriously",
    "Nugatory", "Nurturance", "Obfuscatingly", "Obstreperous", "Obtrusively", "Odiousness", "Officiousness",
    "Oligarchic", "Onerously", "Opalescent", "Operability", "Opportunely", "Opprobriously", "Optimization",
    "Ornamentally", "Ornithology", "Orthographical", "Orthographically", "Ostentatiously", "Outlandishly",
    "Outspokenness", "Overambitious", "Overestimations", "Overindulgence", "Overweening", "Oxymoronic",
    "Palatability", "Palliative", "Panegyrical"]

english_words_c2 += [
    "Paradoxically", "Parochialism", "Particularity", "Pathological", "Pellucidly", "Penetrative", "Penumbral",
    "Perambulate", "Perambulation", "Perdurability", "Peremptorily", "Perfidiously", "Perfunctorily", "Peripatetic",
    "Periphrastic", "Perpetrator", "Perplexingly", "Persnickety", "Personification", "Perspicuously", "Pertinacious",
    "Pertinently", "Pervicacious", "Petrichor", "Phenomenally", "Philanthropic", "Philistinism", "Phlegmatic",
    "Piquant", "Plenipotentiary", "Plenitude", "Plenitudinous", "Plenteous", "Polyglot", "Polyphony", "Polyvalent",
    "Positivistic", "Posthumous", "Postprandial", "Potentate", "Pragmatism", "Pragmatist", "Precipitancy",
    "Predestination", "Preposterously", "Prescient", "Presumptuous", "Pretentiously", "Pristinely", "Probity",
    "Procrastination", "Profoundly", "Profundity", "Progenitor", "Prognostication", "Prohibitive", "Propinquity",
    "Propitiation", "Prosaically", "Prosperously", "Prostration", "Protuberance", "Prowess", "Prudential",
    "Punctiliousness", "Punctuality", "Puniness", "Pyrrhic", "Quandary", "Quintessence", "Quizzically", "Raconteur",
    "Rambunctious", "Rapidity", "Rapscallion", "Ratification", "Recapitulation", "Recondite", "Recrimination",
    "Redemptive", "Redolence", "Refulgence", "Refutation", "Regurgitate", "Rejuvenescence", "Rejuvenation",
    "Relativism", "Relinquishment", "Remonstrance", "Renaissance", "Reparations", "Repertoire", "Replete",
    "Reprehensibility", "Reproachfully", "Reproof", "Reprove", "Repugnant", "Requiem", "Rescind", "Resonant",
    "Resplendent", "Restitution", "Resurgence", "Reticulation", "Reticulocytosis", "Retrogression", "Revivify",
    "Reviviscence", "Revulsion", "Rhapsodical", "Ribald", "Rigorousness", "Riposte", "Ritualistic", "Robustious",
    "Ruminative", "Rutherfordium", "Sacrosanct", "Sagaciousness", "Salubriously", "Salutary", "Salutatorian",
    "Sanctimonious", "Sanguinity", "Sapiential", "Saprophytic", "Sardonically", "Saturnine", "Scintillation",
    "Scrupulousness", "Scurrilous", "Sequester", "Serendipitous", "Sereneness", "Sesquipedalian", "Sibylline",
    "Simpatico", "Simultaneity", "Sinecure", "Singularity", "Sinisterly", "Sobriquet", "Solicitous", "Solicitude",
    "Soliloquy", "Solipsism", "Soporiferous", "Spectroscopy", "Spectroscopic", "Spectacularly", "Splenetic",
    "Splendiferous", "Spontaneity", "Sprightliness", "Stalwartness", "Staunchness", "Steadfastness", "Stentorian",
    "Stigmatization", "Stimulative", "Stipendiary", "Stipulation", "Stoicism", "Stratagem", "Stultifying",
    "Subconsciousness", "Substantive", "Subterfuge", "Subterranean", "Sufficiency", "Suggestiveness", "Supercilious",
    "Superficiality", "Supernumerary", "Supersede", "Supersession", "Supervenient", "Supposititious", "Surfeited",
    "Surreptitiously", "Surreptitiousness", "Sustentation", "Sycophantically", "Sycophantish", "Symphonic", "Synchronicity",
    "Synchronism", "Tactician", "Tactility", "Taciturnity", "Tactfulness", "Tantamount", "Tarradiddle", "Tatterdemalion",
    "Teetotaler", "Tendentiousness", "Terpsichorean", "Terrestrial", "Territoriality", "Thaumaturgy", "Theatricality",
    "Theocracy", "Theosophy", "Threnody", "Thrivingly", "Tidiness", "Tigerish", "Tirade", "Titillation", "Toponym",
    "Torchbearer", "Touchiness", "Traducement", "Transcendental", "Transcendentally", "Transfiguration", "Transmutation",
    "Travesty", "Treacherously", "Tremendously", "Trenchantly", "Triangulation", "Tribulation", "Trifurcation", "Triskaidekaphobia",
    "Truculence", "Trumpery", "Trustworthiness", "Turgidness", "Turbidity", "Turpitude", "Ubiquitously", "Umbrage", "Unassumingness",
    "Unattainable", "Unavailing", "Unbending", "Unconscionable", "Uncontroversial", "Unconventionality", "Uncorrupted",
    "Undeniably", "Underling", "Underpinning", "Underscore", "Undulate", "Unencumbered", "Unequivocal", "Unexceptionable",
    "Unflappable", "Unfounded", "Unhallowed", "Unimpeachable", "Unimpeachably", "Uninhibitedly", "Uninhibitedness", "Uninvolved",
    "Universalism", "Unmitigated", "Unobstructed", "Unpalatable", "Unpretentious", "Unprepossessing", "Unpropitious", "Unqualified",
    "Unremitting", "Unsparing", "Unswerving", "Untarnished", "Untiring", "Unwavering", "Uprightness", "Uproariously", "Uproariousness",
    "Uprooted", "Uprootment", "Usurp", "Usurper", "Utilitarianism", "Uxorious", "Vacillant", "Vacillation", "Vacuousness", "Vainglorious",
    "Valediction", "Valedictorian", "Valetudinarian", "Vehemency", "Vegetarianism", "Venerable", "Venial", "Venturous", "Veracious",
    "Verdancy", "Verifiably", "Verisimilitude", "Vernacular", "Verterbrate", "Vertiginous", "Vicarious", "Vicissitudes", "Vilipend",
    "Vindication", "Virtuosity", "Virtuousness", "Virulently", "Visceral", "Vitriolic", "Vituperate", "Vituperative", "Vivaciousness",
    "Vivacity", "Vociferousness", "Voluptuous", "Voraciousness", "Voyeuristic", "Vulcanization", "Vulnerary", "Vulnerable", "Waggishness",
    "Wardship", "Warmongering", "Warrantable", "Warranted", "Wealthiness", "Welterweight", "Whimsically", "Whimsicalness", "Wholesomeness",
    "Wholesale", "Wistfulness", "Witless", "Witticism", "Wittily", "Woebegone", "Wonderstruck", "Wonted", "Wooingly", "Woolliness",
    "Worshipfulness", "Wretchedness", "Wrigglingly", "Xenogeneic", "Xenophilic", "Xerophilous", "Yesteryear", "Yieldingness", "Youthful",
    "Zephyrous", "Zeugma", "Zestful", "Zoophagous"
]
english_words_c2 = [word.lower() for word in english_words_c2]



dictm['C2'] = english_words_c2
for key in dictm.keys():
    dictm[key] = list(set(dictm[key]))
# исходный словарь
original_dict = dictm

# Создаем новый упорядоченный словарь, основанный на исходном
ordered_dict = OrderedDict(original_dict)

# Меняем порядок ключей вручную
new_order = ['A1', 'A2', 'B1', 'B2', 'C1', 'C2']
level_dict = OrderedDict((key, ordered_dict[key]) for key in new_order)

In [955]:
# df['predicted_level'] = df['Subtitles'].apply(f, o_dict = level_dict) 

In [956]:
df

,Level,Subtitles
0,B1,fix & sync bozxphd enjoy flick clang drawer cl...
1,B1,"hey ill right , cameron go nine school year , ..."
2,B2,resync xenzainef retail help he due list two m...
3,B2,sync correct mrcjnthn get black eye open wide ...
4,A2,"oh , come land faraway place caravan camel roa..."
...,...,...
1729,C2,seaworld suffer collaps profit custom desert c...
1730,C2,"may know steve job even tie shoelac , averag s..."
1731,C2,"long time ago , cinema audienc transport galax..."
1732,C2,"may yet live age fli car , predict film back f..."


In [26]:
def f(text, o_dict):
    
    # Preprocess the text (if needed)
    # You can use the same preprocess_text function mentioned earlier, or customize it based on your requirements.
    
    text = text.replace('\ufeff', '')
    text = text.lower()
    text = re.sub(r"[,:;()\-]", " ", text)  # Override commas, colons, etc to spaces/
    text = re.sub(r"[\.!?]", ".", text)  # Change all terminators like ! and ? to "."
    text = re.sub(r"^\s+", "", text)  # Remove white space
    text = re.sub(r"[ ]*(\n|\r\n|\r)[ ]*", " ", text)  # Remove new lines
    text = re.sub(r'\d+:\d+:\d+,\d+|\d+|<[^>]*>|-|–|>|%|[’\':;♪(){}\[\]]', '', text)  # Remove timecodes, digits, HTML tags, special characters, brackets, exclamation marks, question marks, colons, and other similar characters
    text = re.sub(r"([\.])[\. ]+", ".", text)  # Change all ".." to "."
    text = re.sub(r"[ ]*([\.])", ". ", text)  # Normalize all "."`
    text = re.sub(r"\s+", " ", text)  # Remove multiple spaces
    text = re.sub(r"\s+$", "", text)  # Remove trailing spaces
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)
    text = text.replace('"', '')  # Remove smart quotes
    text = ''.join(char for char in text if ord(char) < 128)
    text = text.replace('\ufeff', '')
    text = text.replace('.', '')
    
    
    words = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]

    # Join the processed words back into a single string
    text = ' '.join(words)

    
    
#     stemmer = PorterStemmer()
    
#     # Tokenize the text into individual words
#     words = word_tokenize(text)
    
#     # Apply stemming to each word and store the results in a list
#     stemmed_words = [stemmer.stem(word) for word in words]
    
#     # Join the stemmed words back into a single string
#     text = ' '.join(stemmed_words)
    
    
#     words = word_tokenize(text)
#     lemma = nltk.wordnet.WordNetLemmatizer()
#     lemmed_words = [lemma.lemmatize(word) for word in words]
#     text = ' '.join(lemmed_words)
    
    
    # Tokenize the text into individual words
    words = text.split()
    words = [word.lower() for word in words]
#     print(words)
    # Initialize a dictionary to store the sum of weights for each difficulty level
    difficulty_weight_sum = {level: 0 for level in o_dict.keys()}
    
    weights = {'A1':1, 'A2':1, 'B1':1, 'B2':1, 'C1':1, 'C2':1}  
    
    # Calculate the sum of weights for each difficulty level in the text
    for word in words:
        for level, word_list in o_dict.items():
            if word in word_list:
                # Assign the weight as the difficulty level (higher level -> bigger weight)
#                 weight = int(level[1:])
                difficulty_weight_sum[level] += weights[level]
#             else:
#                 g = 1
#                 for level in o_dict.keys():
#                     difficulty_weight_sum[level] += g
#                     g+=1
    # Normalize the weights to obtain the final difficulty level probabilities
    total_weight = sum(difficulty_weight_sum.values())
    features = {level: (weight_sum / total_weight) + 1 for level, weight_sum in difficulty_weight_sum.items()}
    return list(features.values())
    
